In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('firefox')

2023-01-01 16:34:03,884 INFO: ====== WebDriver manager ======
2023-01-01 16:34:04,233 INFO: Get LATEST geckodriver version for 108.0 firefox
2023-01-01 16:34:04,889 INFO: Driver [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32\geckodriver.exe] found in cache


**Scrape New Completed Games and Format Them**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            18 non-null     int64         
 1   GAME_DATE_EST   18 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  18 non-null     int64         
 3   PTS             18 non-null     int64         
 4   FG_PCT          18 non-null     float64       
 5   FG3_PCT         18 non-null     float64       
 6   FT_PCT          18 non-null     float64       
 7   REB             18 non-null     int64         
 8   AST             18 non-null     int64         
 9   TEAM_ID         18 non-null     object        
 10  GAME_ID         18 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 1.7+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-31,0,130,53.8,47.1,96.6,39,23,1610612746,22200538,131,52.3,36.8,86.2,36,28,1610612754,2022
1,2022-12-31,1,123,60.0,38.2,73.7,42,26,1610612751,22200539,106,39.6,26.8,90.5,45,28,1610612766,2022
2,2022-12-31,1,103,45.9,34.2,63.2,50,20,1610612739,22200540,102,48.1,33.3,70.8,40,14,1610612741,2022
3,2022-12-31,1,126,58.1,41.2,57.1,26,21,1610612742,22200542,125,55.3,42.1,71.9,48,28,1610612759,2022
4,2022-12-31,1,108,38.9,29.4,71.9,46,20,1610612752,22200541,88,37.7,30.4,69.6,52,20,1610612745,2022
5,2022-12-31,1,115,46.5,32.4,70.6,56,29,1610612755,22200545,96,38.0,33.3,63.6,50,16,1610612760,2022
6,2022-12-31,1,116,48.3,37.0,71.4,52,21,1610612765,22200544,104,43.4,29.4,84.6,39,17,1610612750,2022
7,2022-12-31,0,101,35.2,26.7,80.6,46,16,1610612740,22200543,116,51.1,17.2,77.3,52,26,1610612763,2022
8,2022-12-31,1,126,50.6,33.3,76.7,37,32,1610612748,22200546,123,43.2,42.5,81.8,49,23,1610612762,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(driver)


print(matchups)
print(game_ids)


[['1610612758', '1610612763'], ['1610612764', '1610612749'], ['1610612738', '1610612743']]
['22200547', '22200548', '22200549']


**Close Webdriver**

In [6]:
driver.close() 

**Create Rows for Today's Games with Empty Stats**

In [7]:
# append today's matchups to the new games dataframe


df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': int(game_ids[i]),                       
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612763,22200547,0.0,0.0,0.0,0.0,0.0,0.0,1610612758,2022
1,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612749,22200548,0.0,0.0,0.0,0.0,0.0,0.0,1610612764,2022
2,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612743,22200549,0.0,0.0,0.0,0.0,0.0,0.0,1610612738,2022


**Access Feature Store**

In [8]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [9]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2023-01-01 16:36:06,627 INFO: USE `nba_predictor_featurestore`
2023-01-01 16:36:07,026 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22607,2022-12-11,20000003,1610612752,1610612758,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
22608,2022-12-10,22200394,1610612750,1610612757,2022,118,54.187500,75.000000,43.500000,22,28,124,51.812500,89.312500,38.187500,20,45,0
22609,2022-12-10,22200389,1610612746,1610612764,2022,114,42.593750,93.812500,39.593750,22,42,107,43.500000,89.500000,47.093750,24,50,1
22610,2022-12-11,20000005,1610612755,1610612766,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0


**Convert Feature Names back to original mixed case**

In [11]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22607,2022-12-11,20000003,1610612752,1610612758,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
22608,2022-12-10,22200394,1610612750,1610612757,2022,118,54.187500,75.000000,43.500000,22,28,124,51.812500,89.312500,38.187500,20,45,0
22609,2022-12-10,22200389,1610612746,1610612764,2022,114,42.593750,93.812500,39.593750,22,42,107,43.500000,89.500000,47.093750,24,50,1
22610,2022-12-11,20000005,1610612755,1610612766,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [12]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results
# (these will be games where single games stats will == 0)

df_old.set_index('GAME_ID', inplace=True)
df_old.update(df_new.set_index('GAME_ID'))
df_old.reset_index()  

#df_pending = df_old[df_old['PTS_Home'] == 0] #games that are pending final results

#df_pending.update(df_new)

df_old

,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
GAME_ID,,,,,,,,,,,,,,,,,
21700374,2017-12-08,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
21200874,2013-03-01,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
20500210,2005-11-30,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
21800395,2018-12-10,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
20600946,2007-03-12,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000003,2022-12-11,1610612752,1610612758,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
22200394,2022-12-10,1610612750,1610612757,2022,118,54.187500,75.000000,43.500000,22,28,124,51.812500,89.312500,38.187500,20,45,0
22200389,2022-12-10,1610612746,1610612764,2022,114,42.593750,93.812500,39.593750,22,42,107,43.500000,89.500000,47.093750,24,50,1


**Add Today's Matchups for Feature Engineering**

In [13]:
df_combined = pd.concat([df_old, df_today], ignore_index = True)
df_combined

,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,GAME_ID
0,2017-12-08 00:00:00,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,NaN
1,2013-03-01 00:00:00,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,NaN
2,2005-11-30 00:00:00,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0,NaN
3,2018-12-10 00:00:00,1610612749,1610612739,2018,108.0,0.437988,0.817871,0.416992,22.0,58.0,92.0,0.375000,0.666992,0.333008,24.0,46.0,1.0,NaN
4,2007-03-12 00:00:00,1610612756,1610612745,2006,103.0,0.500000,0.727051,0.600098,18.0,50.0,82.0,0.385986,0.722168,0.262939,13.0,36.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22610,2022-12-11 00:00:00,1610612755,1610612766,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN
22611,2022-12-11 00:00:00,1610612745,1610612749,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN
22612,2023-01-01,1610612763,1610612758,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,22200547
22613,2023-01-01,1610612749,1610612764,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,22200548


**Data Processing**

In [14]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

TypeError: '>' not supported between instances of 'str' and 'int'

**Feature Engineering**

In [ ]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


**Insert New Data into Feature Group**

In [ ]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

#rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})